In [1]:
import pickle
X_hr,y_stress,y_participant,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr.p','rb'))

X_hr = X_hr.reshape(-1,30,1)

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances

train_participant = y_participant.copy()

np.unique(y_participant),train_participant.shape

train_x = X_hr
train_y = y_participant

train_x, test_x, train_y, test_y,participant_ids_train, participant_ids_test = train_test_split(X_hr,
                                                            train_participant,
                                                            y_participant,
                                                            test_size = 0.2,
                                                            random_state=41,
                                                            stratify=y_participant)
train_x, val_x, train_y, val_y, participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=41,
                                                            stratify=participant_ids_train)
train_x.shape,test_x.shape,val_x.shape,train_y.shape,test_y.shape,val_y.shape,participant_ids_train.shape

def get_model(input_shape=(30,1),act='relu',loss="categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Input(shape=input_shape))
    model.add(Reshape(input_shape))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(300,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes//2,activation=None))
    model.add(Dense(n_classes,activation=None))
#     model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
#     model.compile(loss=loss,optimizer=opt,metrics=['acc'])

    return model

n_classes = 1000
model = get_model(input_shape=(30,1),n_classes=n_classes)

model.compile(optimizer='adam',loss=tfa.losses.TripletSemiHardLoss())

model.summary()

model.layers[0]

from keras.models import load_model
filepath = '../model_files/base_cnn_60_seconds_ppg_hr_wesad.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

model = load_model(filepath)
train_embeddings = model.predict(train_x)

In [15]:
def give_rank_accuracy(train_embeddings,train_y,rank = 10):
    distance_matrix = euclidean_distances(train_embeddings)
    distance_matrix_2d = np.zeros((distance_matrix.shape[0],
                               distance_matrix.shape[0],
                               2))
    distance_matrix_2d[:,:,0] = distance_matrix
    train_y_matrix = np.concatenate([train_y.reshape(1,-1)]*distance_matrix.shape[0])
    distance_matrix_2d[:,:,1] = train_y_matrix
    match = 0
    for i in range(distance_matrix.shape[0]):
        row = distance_matrix_2d[i]
        row = row[row[:,0].argsort(),:]
        if rank==1:
            if train_y[i]==row[1,1]:
                match+=1
        else:
            row = set(row[1:(rank+1)][:,1])
            if train_y[i] in row:
                match+=1
    return match/distance_matrix.shape[0]

In [16]:
rank_1 = give_rank_accuracy(train_embeddings,train_y,rank = 1)
rank_5 = give_rank_accuracy(train_embeddings,train_y,rank = 5)
rank_10 = give_rank_accuracy(train_embeddings,train_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.999579383311224 0.9998705794803766 0.9999029346102825


In [17]:
test_embeddings = model.predict(test_x)

In [18]:
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.9664570230607966 0.9811320754716981 0.9842767295597484


In [19]:
test_x,y_stress,test_y,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr_dalia.p','rb'))

In [20]:
test_x = test_x.reshape(-1,30,1)

In [21]:
test_embeddings = model.predict(test_x)
rank_1 = give_rank_accuracy(test_embeddings,test_y,rank = 1)
rank_5 = give_rank_accuracy(test_embeddings,test_y,rank = 5)
rank_10 = give_rank_accuracy(test_embeddings,test_y,rank = 10)
print(rank_1,rank_5,rank_10)

0.9881862404447533 0.990271021542738 0.9916608756080612


In [22]:
from sklearn.metrics import classification_report,confusion_matrix

In [23]:
print(classification_report(train_y,train_y_pred))
print(confusion_matrix(train_y,train_y_pred))

NameError: name 'train_y_pred' is not defined